# sacfc - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 22, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189' 
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 31 # sacfc

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrMLS].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,31,457930477,None,2016,Mini/Flex Plan,171.0,2016-09-17,1.0,2,TRUE,23,23,None,2,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,4,0,0,0,2,0,0,Unknown,1,1,None,None,None,None,0.0,0,17,None,0
1,31,457962304,None,2016,Mini/Flex Plan,45.0,2016-09-17,1.5,11,TRUE,32,32,None,3,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,8.0,0,17,None,0
2,31,457981813,None,2016,Mini/Flex Plan,30.0,1970-01-01,0.0,2,TRUE,122,122,None,0,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,2,0,0,0,2,0,0,Unknown,1,1,None,None,None,None,0.0,0,17,None,0
3,31,458000321,None,2016,Mini/Flex Plan,525.0,2016-09-17,1.0,8,TRUE,50,50,None,4,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,17,None,0
4,31,457922377,None,2016,Mini/Flex Plan,350.0,2016-09-17,0.0,16,TRUE,16,16,None,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,17,None,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11655 entries, 0 to 11654
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            11655 non-null  int64  
 1   dimCustomerMasterId     11655 non-null  int64  
 2   customerNumber          767 non-null    object 
 3   year                    11655 non-null  int64  
 4   productGrouping         11655 non-null  object 
 5   totalSpent              11655 non-null  float64
 6   recentDate              11655 non-null  object 
 7   attendancePercent       11655 non-null  float64
 8   renewedBeforeDays       11655 non-null  int64  
 9   isBuyer                 11655 non-null  object 
 10  source_tenure           11655 non-null  int64  
 11  tenure                  11655 non-null  int64  
 12  distToVenue             0 non-null      object 
 13  totalGames              11655 non-null  int64  
 14  recency                 11655 non-null

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(11655, 32)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11655 entries, 0 to 11654
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    11655 non-null  int64  
 1   totalSpent              11655 non-null  float64
 2   attendancePercent       11655 non-null  float64
 3   renewedBeforeDays       11655 non-null  int64  
 4   source_tenure           11655 non-null  int64  
 5   tenure                  11655 non-null  int64  
 6   totalGames              11655 non-null  int64  
 7   recency                 11655 non-null  int64  
 8   missed_games_1          11655 non-null  int64  
 9   missed_games_2          11655 non-null  int64  
 10  missed_games_over_2     11655 non-null  int64  
 11  click_link              11655 non-null  int64  
 12  open_email              11655 non-null  int64  
 13  send_email              11655 non-null  int64  
 14  openToSendRatio         11655 non-null

### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,year,totalSpent,attendancePercent,renewedBeforeDays,source_tenure,tenure,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,open_email,send_email,openToSendRatio,clickToSendRatio,internal_note,phonecall,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
year,1.00,-0.12,0.11,0.37,0.44,0.41,-0.35,-0.30,-0.21,-0.33,-0.24,0.38,0.47,0.63,0.36,0.27,-0.03,0.01,0.89,0.72,-0.44
totalSpent,-0.12,1.00,-0.04,-0.06,-0.01,-0.00,0.21,0.02,0.03,0.04,0.16,-0.04,-0.04,-0.06,-0.06,-0.06,0.02,0.04,-0.16,-0.03,0.06
attendancePercent,0.11,-0.04,1.00,-0.16,0.08,0.07,0.19,-0.35,0.02,-0.07,-0.41,0.10,0.11,0.02,0.16,0.15,0.03,0.01,0.17,-0.09,0.15
renewedBeforeDays,0.37,-0.06,-0.16,1.00,0.45,0.42,0.26,0.02,0.16,0.07,0.16,0.30,0.36,0.56,0.19,0.17,-0.01,0.05,0.15,0.67,-0.41
source_tenure,0.44,-0.01,0.08,0.45,1.00,0.99,0.06,-0.11,0.02,-0.07,-0.02,0.26,0.31,0.41,0.22,0.19,0.00,0.01,0.35,0.42,-0.16
tenure,0.41,-0.00,0.07,0.42,0.99,1.00,0.06,-0.11,0.02,-0.06,-0.02,0.25,0.29,0.39,0.20,0.18,0.01,0.02,0.32,0.38,-0.14
totalGames,-0.35,0.21,0.19,0.26,0.06,0.06,1.00,-0.12,0.49,0.31,-0.00,0.14,0.11,0.12,0.03,0.09,0.04,0.10,-0.51,0.06,0.15
recency,-0.30,0.02,-0.35,0.02,-0.11,-0.11,-0.12,1.00,-0.07,0.06,0.42,-0.13,-0.15,-0.13,-0.16,-0.14,-0.01,-0.03,-0.32,-0.09,-0.05
missed_games_1,-0.21,0.03,0.02,0.16,0.02,0.02,0.49,-0.07,1.00,0.15,-0.14,0.05,0.03,0.09,-0.02,0.02,0.02,0.07,-0.32,-0.01,0.10
missed_games_2,-0.33,0.04,-0.07,0.07,-0.07,-0.06,0.31,0.06,0.15,1.00,-0.02,-0.07,-0.08,-0.11,-0.09,-0.06,0.02,0.01,-0.35,-0.08,0.09


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (9324, 21)
Unseen Data For Predictions: (2331, 21)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "totalSpent",
        "attendancePercent",
        "renewedBeforeDays",
        "source_tenure",
        "tenure",
        "recency",
        "totalGames",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "click_link",
        "open_email",
        "send_email",
        "openToSendRatio",
        "clickToSendRatio",
        "internal_note",
        "phonecall",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,3454
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(9324, 21)"
5,Missing Values,False
6,Numeric Features,19
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


(0       1
 1       1
 2       1
 3       0
 4       0
        ..
 9319    0
 9320    0
 9321    1
 9322    1
 9323    0
 Name: isNextYear_Buyer, Length: 9324, dtype: int64,
 {'parameter': 'Hyperparameters',
  'auc': 'AUC',
  'confusion_matrix': 'Confusion Matrix',
  'threshold': 'Threshold',
  'pr': 'Precision Recall',
  'error': 'Prediction Error',
  'class_report': 'Class Report',
  'rfe': 'Feature Selection',
  'learning': 'Learning Curve',
  'manifold': 'Manifold Learning',
  'calibration': 'Calibration Curve',
  'vc': 'Validation Curve',
  'dimension': 'Dimensions',
  'feature': 'Feature Importance',
  'feature_all': 'Feature Importance (All)',
  'boundary': 'Decision Boundary',
  'lift': 'Lift Chart',
  'gain': 'Gain Chart',
  'tree': 'Decision Tree',
  'ks': 'KS Statistic Plot'},
 'clf-default-name',
 -1,
 Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=False, 

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9057,0.9577,0.9434,0.8973,0.9196,0.8059,0.8077,0.6360
lr,Logistic Regression,0.8335,0.8926,0.9223,0.8123,0.8636,0.6521,0.6614,0.5750


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9008,0.9584,0.9321,0.8984,0.9149,0.7961,0.7968
1,0.9196,0.9580,0.9554,0.9085,0.9314,0.8344,0.8360
2,0.9021,0.9530,0.9601,0.8796,0.9181,0.7972,0.8019
3,0.9088,0.9484,0.9484,0.8978,0.9224,0.8122,0.8140
4,0.8981,0.9584,0.9507,0.8804,0.9142,0.7893,0.7928
5,0.9249,0.9684,0.9343,0.9343,0.9343,0.8468,0.8468
6,0.9142,0.9579,0.9554,0.9004,0.9271,0.8231,0.8252
7,0.9182,0.9708,0.9507,0.9101,0.9300,0.8318,0.8330
8,0.8834,0.9461,0.9343,0.8709,0.9015,0.7590,0.7619
9,0.8872,0.9573,0.9131,0.8922,0.9026,0.7688,0.7691


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?